<details><summary> </summary>

# Skip notebook test

</details>

In [ ]:
import os
import pandas as pd
import notebook_utils.notebook_helpers as utils
from cuopt_thin_client import CuOptServiceClient

# Intra-factory Transport
## Capacitated Pickup and Delivery Problem with Time Windows

Factory automation allows companies to raise the quality and consistency of manufacturing processes while also allowing human workers to focus on safer, less repetitive tasks that have higher cognitive and creative demands.

In this scenario we have a set of intra-factory transport orders to move products at various stages in the assembly process from one processing station to another. Each station represents a particular type of manufacturing process and a given product may need to visit each processing station more than once. Multiple autonomous mobile robots (AMRs) with a fixed capacity will execute pickup and delivery orders between target locations, all with corresponding time_windows.

### Problem Details:
- 4 Locations each with an associated demand
    - 1 Start Location for AMRs

    - 3 Process Stations

- 3 AMRs with associated capacity

- Hours of operation

In [ ]:
factory_open_time = 0
factory_close_time = 100

# Initialize cuOpt Service Client and cuOpt Problem Data

In [ ]:
cuopt_client_id = os.environ["CUOPT_CLIENT_ID"]
cuopt_client_secret = os.environ["CUOPT_CLIENT_SECRET"]


cuopt_service_client = CuOptServiceClient(
    client_id=cuopt_client_id,
    client_secret=cuopt_client_secret,
    )

cuopt_problem_data = {}

![waypoint_graph.png not found](./notebook_utils/images/waypoint_graph.png "Waypoint Graph")

### Set location names

In [ ]:
location_names = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]

### Waypoint Graph

#### Compressed Sparse Row (CSR) representation of above weighted waypoint graph.
For details on the CSR encoding of the above graph see the [cost_matrix_and_waypoint_graph_creation.ipynb](cost_matrix_and_waypoint_graph_creation.ipynb) notebook.

In [ ]:
offsets = [0, 1, 3, 7, 9, 11, 13, 15, 17, 20, 22]
edges =   [2, 2, 4, 0, 1, 3, 5, 2, 6, 1, 7, 2, 8, 3, 9, 4, 8, 5, 7, 9, 6, 8]
weights = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 1, 2, 1, 2, 2, 1, 2]

#### Select specific waypoints in the graph as target locations
In this case we would like the AMRs to begin from waypoint 0 and service locations 4, 5, and 6.

In [ ]:
# Setup service locations
target_locations       = [0, 4, 5, 6]

### Transport Orders
Setup Transport Order Data

The transport orders dictate the movement of parts from one area of the factory to another.  In this example nodes 4, 5, and 6 represent the processing stations that parts must travel between and deliveries to node 0 represent the movement of parts off the factory floor.

In [ ]:
transport_order_data = pd.DataFrame({
    "pickup_location":       [4,  5,  6,  6,  5,  4],
    "delivery_location":     [5,  6,  0,  5,  4,  0],
    "order_demand":          [1,  1,  1,  1,  1,  1],
    "earliest_pickup":       [0,  0,  0,  0,  0,  0],
    "latest_pickup":         [10, 20, 30, 10, 20, 30],
    "pickup_service_time":   [2,  2,  2,  2,  2,  2],
    "earliest_delivery":     [0,  0,  0,  0,  0,  0],
    "latest_delivery":       [45, 45, 45, 45, 45, 45],
    "delivery_service_time": [2,  2,  2,  2,  2,  2]
})
transport_order_data

### Set Waypoint Graph

cuOpt will use this waypoint graph along with task locations and vehicle locations to determine cost matrix internally from one location to another. 

In [ ]:
graph_data = {
    "edges": edges,
    "offsets": offsets,
    "weights": weights,
}

cuopt_problem_data["cost_waypoint_graph_data"] = {
        "waypoint_graph": {
            "0": graph_data
        }
    }

### Set Order/Task data


#### Process Order locations

Order locations, pickup and delivery pairs are processed and created to be digested bu cuOpt

In [ ]:
cuopt_problem_data["task_data"] = {}
pickup_order_locations = transport_order_data['pickup_location']
delivery_order_locations = transport_order_data['delivery_location']
order_locations = pd.concat([pickup_order_locations, delivery_order_locations], ignore_index=True)

cuopt_problem_data["task_data"]["task_locations"] = order_locations.to_list()
print(order_locations)


#### Process demand data

From the perspective of the cuOpt solver_settings, each distinct transaction (pickup order or delivery order) are treated separately with demand for pickup denoted as positive and the corresponding delivery treated as negative demand.

In [ ]:
# This is the number of parts that needs to be moved
raw_demand = transport_order_data["order_demand"]

# When dropping off parts we want to remove one unit of demand from the robot
drop_off_demand = raw_demand * -1

# Create pickup and delivery demand
order_demand = pd.concat([raw_demand, drop_off_demand], ignore_index=True)

# Add demand to the task data
cuopt_problem_data["task_data"]["demand"] = [order_demand.to_list()]
print(order_demand)

#### Process task time windows

In [ ]:
# create earliest times
order_time_window_earliest = pd.concat([transport_order_data["earliest_pickup"], transport_order_data["earliest_delivery"]], ignore_index=True)

# create latest times
order_time_window_latest = pd.concat([transport_order_data["latest_pickup"], transport_order_data["latest_delivery"]], ignore_index=True)

# create service times
order_service_time = pd.concat([transport_order_data["pickup_service_time"],transport_order_data["delivery_service_time"]], ignore_index=True)

# add time window constraints
cuopt_problem_data["task_data"]["task_time_windows"] = list(zip(order_time_window_earliest.to_list(),
                                                                order_time_window_latest.to_list()))
cuopt_problem_data["task_data"]["service_times"] = order_service_time.to_list()

#### Mapping pickups to deliveries

In [ ]:
# IMPORTANT NOTE : pickup and delivery pairs are indexed into the order locations array.
npair_orders = int(len(order_locations)/2)
pickup_order_ids = pd.Series([i for i in range(npair_orders)])
delivery_order_ids = pd.Series([i + npair_orders for i in range(npair_orders)])

# add pickup and delivery pairs.
cuopt_problem_data["task_data"]["pickup_and_delivery_pairs"] = list(zip(pickup_order_ids.to_list(),
                                                                        delivery_order_ids.to_list()))

### Set AMR data

Accumulate AMR fleet data such as its start and end locations, capacity, break/charging times and other details that relate to a vehicle.

In [ ]:
n_robots = 2
cuopt_problem_data["fleet_data"] = {}

# Add start and end locations for AMRs, assuming all AMRs start and end at location 0.
cuopt_problem_data["fleet_data"]["vehicle_locations"] = [[0, 0]] * n_robots

# Add carrying capacity for AMRs, assuming all robots have capacity of 2,
# means, they can carry at the max two items at any point
cuopt_problem_data["fleet_data"]["capacities"] = [[2] * n_robots]

cuopt_problem_data["fleet_data"]["vehicle_time_windows"] = [[factory_open_time, factory_close_time]] * n_robots


### Set Solver Settings

In [ ]:
cuopt_problem_data["solver_config"] = {
        "time_limit": 5
    }

### Get optimized route

In [ ]:
# Solve the problem
solver_response = cuopt_service_client.get_optimized_routes(
    cuopt_problem_data
)

# Process returned data
solver_resp = solver_response["response"]["solver_response"]

if solver_resp["status"] == 0:
    print("Cost for the routing in time: ", solver_resp["solution_cost"])
    print("Vehicle count to complete routing: ", solver_resp["num_vehicles"])
    utils.show_vehicle_routes(solver_resp, location_names)
else:
    print("NVIDIA cuOpt Failed to find a solution with status : ", solver_resp["status"])

#### Waypoint level routes for AMRs

In [ ]:
solver_resp_df = utils.get_solution_df(solver_resp)
unique_robot_ids = solver_resp_df['truck_id'].unique()
all_routes = solver_resp_df

for robot in unique_robot_ids:
    route = all_routes[all_routes['truck_id']==robot]
    unique_target_locs = all_routes[all_routes['truck_id']==robot]['route'].unique()
    
    print(f"Waypoint level route for robot {robot}:\n{all_routes[all_routes['truck_id']==robot]['route']}\n\n")